In [2]:
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print(' Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print(' Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

device: cuda
Tesla P100-PCIE-16GB
 Allocated: 0.0 GB
 Cached:    0.0 GB


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import math
import numpy as np
import time
import json
import os
if not os.path.exists('models'):
    os.mkdir('models')

torch.manual_seed(3)
plt.style.use('dark_background')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.rainbow(np.linspace(0, 1, 5)))

N = 3983
A0 = torch.load('../input/am4-default/A-V4_3.pt', map_location='cuda')
capital_ids = torch.load('../input/am4-default/capital_ids.pt', map_location='cuda')
As = torch.stack([A0, torch.zeros(N, dtype=torch.float32).cuda()])
D = torch.load('../input/am4-default/D.pt', map_location='cuda')

mask = ~torch.eye(N, dtype=bool).cuda()
ys = D[mask].reshape(-1)

mask = mask.view(-1)
i = torch.arange(N, device='cuda').repeat(N)[mask]
j = torch.arange(N, device='cuda').repeat_interleave(N)[mask]

xs = torch.stack((i, j), dim=1) # [[i0, j0], ...]

tensor([[   1,    2,    3,  ..., 3979, 3980, 3981],
        [   0,    0,    0,  ..., 3982, 3982, 3982]], device='cuda:0')


In [53]:
batch = torch.tensor([[3, 999], [4, 3320], [3, 3320]]).cuda()
batch_ij = batch.t()

i, j = batch_ij
icap, jcap = torch.isin(batch_ij, capital_ids).to(torch.int64)
print(i, j)
print(icap, jcap)

print('As', As)
Ai = torch.index_select(As, 1, i)
A = torch.gather(Ai, 0, icap.unsqueeze(0))
print(Ai)
print(A)
# print(iin)

tensor([3, 4, 3], device='cuda:0') tensor([ 999, 3320, 3320], device='cuda:0')
tensor([1, 0, 1], device='cuda:0') tensor([0, 1, 1], device='cuda:0')
As tensor([[ 0.0000, 24.3011,  6.3601,  ..., 13.7397, 13.6036, 23.2647],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<StackBackward0>)
tensor([[26.2364,  7.1814, 26.2364],
        [ 0.0000,  0.0000,  0.0000]], device='cuda:0',
       grad_fn=<IndexSelectBackward0>)
tensor([[0.0000, 7.1814, 0.0000]], device='cuda:0', grad_fn=<GatherBackward0>)


In [ ]:
class DemandModel(nn.Module):
    def __init__(self):
        super(DemandModel, self).__init__()
        self.As = nn.Parameter(As)
        self.Bs = nn.Parameter(torch.zeros((2, N)))
#         self.A = nn.Parameter(torch.rand(N) * 25)
        self.B = nn.Parameter(torch.rand(N) * 3)
#         self.C = nn.Parameter(torch.rand(N) * 25)
#         self.M = nn.Parameter(torch.ones(2*N, dtype=torch.float32))
#         self.K = nn.Parameter(torch.rand(2*N) * 100)

    def forward(self, batch):
        batchin = torch.isin(batch, capital_ids).to(torch.int)
        i, j = batch[:, 0], batch[:, 1]
#         return self.A[i] * self.A[j]
        return self.A[i] * self.A[j] + self.B[i] + self.B[j]
        # return self.A[i] * self.A[j] + self.M[i] + self.M[j]
        # return self.A[i] * self.B[j] + self.A[j] * self.B[i] + self.M[i] + self.M[j]

model = DemandModel().cuda()

batch_size = N // 2
lr = 0.01 / 2 / 2
status_every = 200#N*(N-1) // (batch_size ) / 1000
epochs = 100

print(f'*** STARTING: {batch_size}, {lr}, {epochs}')
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

dataloader = DataLoader(
    TensorDataset(xs, ys),
    batch_size=batch_size,
    shuffle=True
)

start = time.time()
losses = []
last_model = model
for e in range(epochs):
    total_loss = 0
    i = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_x)
        loss = criterion(y_pred, batch_y)
        l1 = .002 * sum([p.abs().sum() for p in model.B])
        loss.backward()
        optimizer.step()
        if i > 0 and i % status_every == 0:
            print(f'{i/len(dataloader)+e:.5f},{math.sqrt(loss)},{l1}')
        
        total_loss += loss.item()
        i += 1

    loss = math.sqrt(total_loss / len(dataloader))
#     print(f'{e:>5} | loss={loss}')

    torch.save(model.A, f'./models/A-V13_{e}.pt')
    torch.save(model.B, f'./models/B-V13_{e}.pt')
#     torch.save(model.C, f'./models/C-V13_{e}.pt')
#     torch.save(model.K, f'./models/M-V12_{e}.pt')

    if loss < 5e-4:
        print(f'*** PAST CRITERON: loss={loss}')
        last_model = model
        break
    if loss < 1 and loss > losses[-1]:
        print(f'*** INCREASING: loss={loss}')
        break
    losses.append(loss)
    last_model = model

print(f'*** DONE: {time.time() - start:.2f}s')
# D_pred = torch.mul(last_model.A.unsqueeze(1), last_model.B) + torch.mul(last_model.A, last_model.B.unsqueeze(1))
# print(D - D_pred)
# print(torch.sum(torch.abs(D - D_pred)))

# plt.plot(losses, label=f'batch_size={batch_size}, lr={lr}')

# plt.legend()
# plt.xlabel('Epoch')
# plt.ylabel('MSE Loss')
# plt.show()

In [ ]:
os.listdir('./models')
import shutil
shutil.copy('./models/A-V7_3.pt', './A-V7_3.pt')
shutil.copy('./models/M-V7_3.pt', './M-V7_3.pt')

# import shutil
# shutil.rmtree('models')

In [ ]:
for f in os.listdir('./models'):
    os.remove(f)
